In [1]:
%matplotlib inline

import pandas as pd 
import numpy as np

In [2]:
df_trips = pd.read_csv("data/generated-marketplace-trips.csv")
df_passengers = pd.read_csv("data/generated-marketplace-passengers.csv")
df_pilots = pd.read_csv("data/generated-marketplace-pilots.csv")
df_planets = pd.read_csv("data/generated-marketplace-planets.csv")

### misc

In [3]:
planet_names = np.append(df_planets["name"].values, "All")
planet_options = [{"label": p, "value": p.lower()} for p in planet_names]

user_types = ["Pilot", "Passenger"]
user_options = [{"label": t, "value": t.lower()} for t in user_types]
df_trips["trip_requested_dt"] = pd.to_datetime(df_trips["trip_requested"], unit="s")
df_trips["trip_requested_month"] = df_trips["trip_requested_dt"].dt.month
trips_months = np.sort(df_trips["trip_requested_month"].unique())
month_options = {str(m): str(m) for m in trips_months}
min_month = trips_months.min()
max_month = trips_months.max()

df_trips_week = df_trips["trip_requested_dt"].dt.week
df_trips_week = df_trips_week.value_counts().sort_index()

### get_month_summary

In [4]:
def get_month_summary(df, month):
    trips, pilots, passengers = None, None, None
    if month in list(df["trip_requested_month"]): # compare with list because `0 in series` returns True
        dff = df[df["trip_requested_month"] == month]
        trips = len(dff)
        pilots = len(dff["pilot"].unique())
        passengers = len(dff["passenger"].unique())
    return np.array([trips, pilots, passengers])

In [5]:
print(get_month_summary(df_trips, -1))
print(get_month_summary(df_trips, 0))
print(get_month_summary(df_trips, 1))
print(get_month_summary(df_trips, 2))
print(get_month_summary(df_trips, 12))
print(get_month_summary(df_trips, 13))

[None None None]
[None None None]
[1 1 1]
[8 7 7]
[4200  100  966]
[None None None]


### generate_summary_table

In [6]:
def generate_summary_table(df, month):
    cm = get_month_summary(df_trips, month)
    pm = get_month_summary(df_trips, month - 1)
    df = pd.DataFrame({
        "Current": cm, "Previous": pm if pm[0] else "-", 
        "M/M Change": np.round((cm * 1.0 / pm), 2) if pm[0] else "-"
    }, index=["Trips", "Pilots", "Passengers"],
    columns=["Current", "Previous", "M/M Change"])
    return df

In [7]:
generate_summary_table(df_trips, 0)

,Current,Previous,M/M Change
Trips,None,-,-
Pilots,None,-,-
Passengers,None,-,-


In [8]:
generate_summary_table(df_trips, 3)

,Current,Previous,M/M Change
Trips,53,8,6.62
Pilots,19,7,2.71
Passengers,44,7,6.29


In [9]:
generate_summary_table(df_trips, 12)

,Current,Previous,M/M Change
Trips,4200,1968,2.13
Pilots,100,92,1.09
Passengers,966,746,1.29
